In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [4]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_full_matrix_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffeled_matrix_train_inputs_other_features_multilabel.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffeled_matrix_test_inputs_other_features_multilabel.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffeled_matrix_validate_inputs_other_features_multilabel.h5', X_validate, y_validate,  other_features = X_other_features_validate)
X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffeled_matrix_train_inputs_other_features_multilabel.h5', other_features=True)
X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffeled_matrix_test_inputs_other_features_multilabel.h5', other_features=True)
X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffeled_matrix_validate_inputs_other_features_multilabel.h5', other_features=True)

In [9]:
print (X_train.shape)
print (X_test.shape)
print (X_validate.shape)
# X_other_features = create_X_features(content)
# print (X_other_features[178200])
decode_position = 38
print (decode_input(X_validate[decode_position], dictionary))
# print (X_other_features_train[0])
print (decode_X_features(feature_dictionary, [X_other_features_train[decode_position]]))
# print (len(X_other_features_train[0]))

(430151, 23, 30)
(52058, 23, 30)
(54222, 23, 30)
najslajši
Agpndi-
Agpndi-


In [11]:
num_examples = len(X_train) # training set size
# nn_input_dim = max_word * len(dictionary) # input layer dimensionality
# nn_output_dim = max_num_vowels * max_num_vowels # output layer dimensionality
nn_output_dim = 11
nn_hdim = 516
batch_size = 16
actual_epoch = 40
num_fake_epoch = 20

# Gradient descent parameters (I picked these by hand)
# epsilon = 1 # learning rate for gradient descent
# reg_lambda = 1 # regularization strength

In [12]:
conv_input_shape=(23, 30)
othr_input = (140, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(43, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(43, (3), padding='same', activation='relu')(x_conv)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)

In [13]:
model = Model(inputs=[conv_input, othr_input], outputs=x)

# epochs = 5
# lrate = 0.1
# decay = lrate/epochs
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Compile model
# keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [14]:
history = model.fit_generator(generate_fake_epoch(X_train, X_other_features_train, y_train, batch_size), X_train.shape[0]/(batch_size * num_fake_epoch), epochs=actual_epoch*num_fake_epoch, validation_data=([X_test, X_other_features_test], y_test))

# model.fit([X_train, X_other_features_train], y_train, validation_data=([X_validate, X_other_features_validate], y_validate), epochs=1, batch_size=16)

Epoch 1/800
1345/1344 [==============================] - 27s - loss: 0.2206 - actual_accuracy: 0.1980 - val_loss: 0.1649 - val_actual_accuracy: 0.4996
Epoch 2/800
1345/1344 [==============================] - 27s - loss: 0.1542 - actual_accuracy: 0.4501 - val_loss: 0.1426 - val_actual_accuracy: 0.5590
Epoch 3/800
1345/1344 [==============================] - 29s - loss: 0.1342 - actual_accuracy: 0.5430 - val_loss: 0.1267 - val_actual_accuracy: 0.6358
Epoch 4/800
1345/1344 [==============================] - 28s - loss: 0.1197 - actual_accuracy: 0.6154 - val_loss: 0.1130 - val_actual_accuracy: 0.6962
Epoch 5/800
1345/1344 [==============================] - 28s - loss: 0.1106 - actual_accuracy: 0.6630 - val_loss: 0.1064 - val_actual_accuracy: 0.7232
Epoch 6/800
1345/1344 [==============================] - 29s - loss: 0.1029 - actual_accuracy: 0.7000 - val_loss: 0.1016 - val_actual_accuracy: 0.7380
Epoch 7/800
1345/1344 [==============================] - 29s - loss: 0.0955 - actual_accuracy:

1345/1344 [==============================] - 31s - loss: 0.0317 - actual_accuracy: 0.9215 - val_loss: 0.0918 - val_actual_accuracy: 0.8494
Epoch 55/800
1345/1344 [==============================] - 30s - loss: 0.0305 - actual_accuracy: 0.9239 - val_loss: 0.0975 - val_actual_accuracy: 0.8423
Epoch 56/800
1345/1344 [==============================] - 31s - loss: 0.0294 - actual_accuracy: 0.9255 - val_loss: 0.0901 - val_actual_accuracy: 0.8475
Epoch 57/800
1345/1344 [==============================] - 32s - loss: 0.0282 - actual_accuracy: 0.9296 - val_loss: 0.0963 - val_actual_accuracy: 0.8484
Epoch 58/800
1345/1344 [==============================] - 31s - loss: 0.0284 - actual_accuracy: 0.9319 - val_loss: 0.0957 - val_actual_accuracy: 0.8448
Epoch 59/800
1345/1344 [==============================] - 30s - loss: 0.0279 - actual_accuracy: 0.9303 - val_loss: 0.0927 - val_actual_accuracy: 0.8487
Epoch 60/800
1345/1344 [==============================] - 31s - loss: 0.0283 - actual_accuracy: 0.930

1345/1344 [==============================] - 30s - loss: 0.0174 - actual_accuracy: 0.9575 - val_loss: 0.1062 - val_actual_accuracy: 0.8631
Epoch 108/800
1345/1344 [==============================] - 31s - loss: 0.0181 - actual_accuracy: 0.9564 - val_loss: 0.1101 - val_actual_accuracy: 0.8646
Epoch 109/800
1345/1344 [==============================] - 31s - loss: 0.0181 - actual_accuracy: 0.9580 - val_loss: 0.1161 - val_actual_accuracy: 0.8570
Epoch 110/800
1345/1344 [==============================] - 32s - loss: 0.0176 - actual_accuracy: 0.9570 - val_loss: 0.1171 - val_actual_accuracy: 0.8628
Epoch 111/800
1345/1344 [==============================] - 31s - loss: 0.0166 - actual_accuracy: 0.9600 - val_loss: 0.1083 - val_actual_accuracy: 0.8620
Epoch 112/800
1345/1344 [==============================] - 31s - loss: 0.0172 - actual_accuracy: 0.9584 - val_loss: 0.1134 - val_actual_accuracy: 0.8554
Epoch 113/800
1345/1344 [==============================] - 31s - loss: 0.0167 - actual_accuracy:

1345/1344 [==============================] - 30s - loss: 0.0128 - actual_accuracy: 0.9698 - val_loss: 0.1235 - val_actual_accuracy: 0.8582
Epoch 161/800
1345/1344 [==============================] - 31s - loss: 0.0130 - actual_accuracy: 0.9678 - val_loss: 0.1234 - val_actual_accuracy: 0.8572
Epoch 162/800
1345/1344 [==============================] - 30s - loss: 0.0124 - actual_accuracy: 0.9711 - val_loss: 0.1154 - val_actual_accuracy: 0.8644
Epoch 163/800
1345/1344 [==============================] - 30s - loss: 0.0128 - actual_accuracy: 0.9688 - val_loss: 0.1133 - val_actual_accuracy: 0.8662
Epoch 164/800
1345/1344 [==============================] - 30s - loss: 0.0130 - actual_accuracy: 0.9684 - val_loss: 0.1203 - val_actual_accuracy: 0.8680
Epoch 165/800
1345/1344 [==============================] - 31s - loss: 0.0128 - actual_accuracy: 0.9694 - val_loss: 0.1227 - val_actual_accuracy: 0.8653
Epoch 166/800
1345/1344 [==============================] - 30s - loss: 0.0122 - actual_accuracy:

1345/1344 [==============================] - 32s - loss: 0.0095 - actual_accuracy: 0.9774 - val_loss: 0.1308 - val_actual_accuracy: 0.8657
Epoch 214/800
1345/1344 [==============================] - 31s - loss: 0.0104 - actual_accuracy: 0.9768 - val_loss: 0.1239 - val_actual_accuracy: 0.8655
Epoch 215/800
1345/1344 [==============================] - 31s - loss: 0.0101 - actual_accuracy: 0.9770 - val_loss: 0.1255 - val_actual_accuracy: 0.8705
Epoch 216/800
1345/1344 [==============================] - 32s - loss: 0.0099 - actual_accuracy: 0.9770 - val_loss: 0.1237 - val_actual_accuracy: 0.8710
Epoch 217/800
1345/1344 [==============================] - 31s - loss: 0.0101 - actual_accuracy: 0.9772 - val_loss: 0.1248 - val_actual_accuracy: 0.8642
Epoch 218/800
1345/1344 [==============================] - 30s - loss: 0.0091 - actual_accuracy: 0.9793 - val_loss: 0.1463 - val_actual_accuracy: 0.8575
Epoch 219/800
1345/1344 [==============================] - 31s - loss: 0.0106 - actual_accuracy:

...

1345/1344 [==============================] - 32s - loss: 0.0056 - actual_accuracy: 0.9878 - val_loss: 0.1446 - val_actual_accuracy: 0.8709
Epoch 585/800
1345/1344 [==============================] - 32s - loss: 0.0050 - actual_accuracy: 0.9891 - val_loss: 0.1531 - val_actual_accuracy: 0.8670
Epoch 586/800
1345/1344 [==============================] - 32s - loss: 0.0049 - actual_accuracy: 0.9900 - val_loss: 0.1526 - val_actual_accuracy: 0.8694
Epoch 587/800
1345/1344 [==============================] - 33s - loss: 0.0054 - actual_accuracy: 0.9897 - val_loss: 0.1443 - val_actual_accuracy: 0.8721
Epoch 588/800
1345/1344 [==============================] - 32s - loss: 0.0052 - actual_accuracy: 0.9882 - val_loss: 0.1292 - val_actual_accuracy: 0.8723
Epoch 589/800
1345/1344 [==============================] - 33s - loss: 0.0048 - actual_accuracy: 0.9891 - val_loss: 0.1531 - val_actual_accuracy: 0.8704
Epoch 590/800
1345/1344 [==============================] - 33s - loss: 0.0058 - actual_accuracy:

1345/1344 [==============================] - 42s - loss: 0.0048 - actual_accuracy: 0.9899 - val_loss: 0.1504 - val_actual_accuracy: 0.8722
Epoch 638/800
1345/1344 [==============================] - 39s - loss: 0.0050 - actual_accuracy: 0.9902 - val_loss: 0.1545 - val_actual_accuracy: 0.8698
Epoch 639/800
1345/1344 [==============================] - 41s - loss: 0.0049 - actual_accuracy: 0.9888 - val_loss: 0.1441 - val_actual_accuracy: 0.8690
Epoch 640/800
1345/1344 [==============================] - 40s - loss: 0.0052 - actual_accuracy: 0.9889 - val_loss: 0.1466 - val_actual_accuracy: 0.8704
Epoch 641/800
1345/1344 [==============================] - 41s - loss: 0.0050 - actual_accuracy: 0.9888 - val_loss: 0.1585 - val_actual_accuracy: 0.8691
Epoch 642/800
1345/1344 [==============================] - 41s - loss: 0.0043 - actual_accuracy: 0.9902 - val_loss: 0.1458 - val_actual_accuracy: 0.8699
Epoch 643/800
1345/1344 [==============================] - 38s - loss: 0.0047 - actual_accuracy:

1345/1344 [==============================] - 33s - loss: 0.0045 - actual_accuracy: 0.9899 - val_loss: 0.1410 - val_actual_accuracy: 0.8800
Epoch 691/800
1345/1344 [==============================] - 32s - loss: 0.0050 - actual_accuracy: 0.9894 - val_loss: 0.1497 - val_actual_accuracy: 0.8673
Epoch 692/800
1345/1344 [==============================] - 33s - loss: 0.0049 - actual_accuracy: 0.9904 - val_loss: 0.1597 - val_actual_accuracy: 0.8714
Epoch 693/800
1345/1344 [==============================] - 33s - loss: 0.0042 - actual_accuracy: 0.9911 - val_loss: 0.1523 - val_actual_accuracy: 0.8697
Epoch 694/800
1345/1344 [==============================] - 35s - loss: 0.0049 - actual_accuracy: 0.9898 - val_loss: 0.1566 - val_actual_accuracy: 0.8683
Epoch 695/800
1345/1344 [==============================] - 34s - loss: 0.0048 - actual_accuracy: 0.9899 - val_loss: 0.1474 - val_actual_accuracy: 0.8733
Epoch 696/800
1345/1344 [==============================] - 32s - loss: 0.0057 - actual_accuracy:

1345/1344 [==============================] - 35s - loss: 0.0043 - actual_accuracy: 0.9908 - val_loss: 0.1529 - val_actual_accuracy: 0.8709
Epoch 744/800
1345/1344 [==============================] - 36s - loss: 0.0055 - actual_accuracy: 0.9883 - val_loss: 0.1559 - val_actual_accuracy: 0.8722
Epoch 745/800
1345/1344 [==============================] - 36s - loss: 0.0043 - actual_accuracy: 0.9906 - val_loss: 0.1581 - val_actual_accuracy: 0.8701
Epoch 746/800
1345/1344 [==============================] - 35s - loss: 0.0043 - actual_accuracy: 0.9906 - val_loss: 0.1437 - val_actual_accuracy: 0.8718
Epoch 747/800
1345/1344 [==============================] - 35s - loss: 0.0044 - actual_accuracy: 0.9905 - val_loss: 0.1617 - val_actual_accuracy: 0.8722
Epoch 748/800
1345/1344 [==============================] - 34s - loss: 0.0052 - actual_accuracy: 0.9886 - val_loss: 0.1614 - val_actual_accuracy: 0.8702
Epoch 749/800
1345/1344 [==============================] - 34s - loss: 0.0049 - actual_accuracy:

1345/1344 [==============================] - 35s - loss: 0.0047 - actual_accuracy: 0.9899 - val_loss: 0.1617 - val_actual_accuracy: 0.8713
Epoch 797/800
1345/1344 [==============================] - 33s - loss: 0.0045 - actual_accuracy: 0.9901 - val_loss: 0.1533 - val_actual_accuracy: 0.8715
Epoch 798/800
1345/1344 [==============================] - 33s - loss: 0.0041 - actual_accuracy: 0.9911 - val_loss: 0.1878 - val_actual_accuracy: 0.8639
Epoch 799/800
1345/1344 [==============================] - 34s - loss: 0.0045 - actual_accuracy: 0.9900 - val_loss: 0.1671 - val_actual_accuracy: 0.8742
Epoch 800/800
1345/1344 [==============================] - 33s - loss: 0.0044 - actual_accuracy: 0.9912 - val_loss: 0.1699 - val_actual_accuracy: 0.8697


In [27]:
name = '40_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [3]:
import matplotlib.pyplot as plt
plt.plot(history.history['actual_accuracy'])
plt.plot(history.history['val_actual_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [25]:
model = load_model('test.h5')

In [6]:
h5f.close()

NameError: name 'h5f' is not defined

In [67]:
# evaluate the model
scores = model.evaluate([X_validate, X_other_features_validate], y_validate)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

108736/108961 [============================>.] - ETA: 0s
acc: 96.57%


In [187]:
pos = 0
print(decode_input(X_validate[pos], dictionary))
# predictions = model.predict([X_validate, X_other_features_validate])
predictions2 = model.predict([X_validate[pos:pos+1], X_other_features_validate[pos:pos+1]])
# print(predictions2)
print(np.round(predictions2)[0])
# predictions = np.round(predictions)
print(y_validate[pos])
# accuracy = sum([1 if np.all(y_validate[i] == predictions[i]) else 0 for i in range(X_validate.shape[0])])/float(X_validate.shape[0])
# print(accuracy)


import keras.backend as K

def act_accuracy(y_true, y_pred):
    return K.mean(K.equal(K.mean(K.equal(K.round(y_true), K.round(y_pred)), axis=-1), 1.0))


def mean_pred2(y_true, y_pred):
    return K.mean(K.equal(K.argmax(y_true, axis=-1), K.argmax(y_pred, axis=-1)))
#     return K.mean(K.all(K.equal(y_true, y_pred), axis=-1))
#     return K.equal(K.round(y_true), K.round(y_pred))

print(mean_pred(y_validate[pos], predictions[pos]).eval())
print(mean_pred(np.array([[ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
                          [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]), 
                np.array([[ 0.,  0.51,  0.,  0.51,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
                          [ 0.,  0.92,  0.,  0.51,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])).eval())
# print(mean_pred(y_validate[pos], predictions[pos]))
# print(mean_pred2(y_validate[pos], predictions[pos]))
print(metrics.categorical_accuracy(y_validate[pos], predictions[pos]))

nerazloènejši
[ 0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
1.0
1.0
Elemwise{Cast{float32}}.0


In [ ]:
predictions = model.predict(X[429145:])

In [21]:
decode_position(predictions[0], max_num_vowels)

[1, 0]

In [23]:
def test_accuracy(predictions, y):
    dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
    num_of_pred = len(predictions)
    num_of_correct_pred = 0
    for i in range(predictions.shape[0]):
        if decode_position(predictions[i], max_num_vowels) == decode_position(y[i], max_num_vowels):
            num_of_correct_pred += 1

    return (num_of_correct_pred/float(num_of_pred)) * 100


print(test_accuracy(predictions, y[429145:]))

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
107287
96.5186835311


In [17]:
predictions.shape

(107287, 121)

In [12]:
print max_num_vowels

11


In [6]:
dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
feature_dictionary = create_feature_dictionary()
def generate_input_from_word(word, max_word, dictionary):
    x = np.zeros((max_word, len(dictionary)))
    j = 0
    for c in list(word):
        index = 0
        for d in dictionary:
            if c == d:
                x[j, index] = 1
                break
            index += 1
        j += 1
    return x

# model = load_model()
# prediction = predict(model, generate_input_from_word('hidrija'))
# print decode_position(prediction[0])

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!


In [6]:
%run ../../../prepare_data.py
# generate_X_and_y(dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels, feature_dictionary)